# Airi
Autor: Gabriel Dornelles Monteiro, abril de 2022. Notebook nº3.5.

Construimos dois modelos nos notebooks anteriores, e como vimos, o processo para construir e treinar os estes modelos cresceu muito do primeiro para o segundo, se torna bastante impráticavel aumentarmos os modelos com o paradigma atual que utilizamos até agora. 

Para isso, iremos fazer o mesmo que os modernos frameworks de Machine Learning fazem, iremos modularizar todo nosso processo, isto é, criaremos módulos que executam o que precisamos, calculam o que deve ser calculado, para que dessa maneira, nosso trabalho seja encaixar estes módulos como encaixamos peças de Lego.

De maneira geral, cada um de nossos blocos terá o seguinte formato:

```py
class AiriLayer:
    def __init__(self):
       pass
  
    def __call__(self, x):
        self.forward(x)
    
    def forward(self, x):
        pass
    
    def backward(self, dout):
        pass
    
    def update(self, **kwargs):
        pass

    def zero_grad(self, **kwargs):
        pass
```
Onde :
- em  ```__init__``` temos um construtor padrão para nossa classe
- Utilizamos o magic method ```__call__``` para passar o método forward, ou seja, ao executarmos algo como:
```py
sample_module = AiriLayer()
sample_module(x)
```
Estaremos efetivamente, executando o método forward da classe.
- Em ```forward``` escreveremos o código que performa a computação de inferência do modelo
- Em ```backward``` escreveremos o código que calcula os gradientes para nosso bloco, e também que ira retornar o gradiente que deve seguir seu fluxo pelo modelo
- Em ```update``` escreveremos o código que irá realizar efetivamente todos os updates dos parâmetros pertencentes a nosso bloco.
- Em ```zero_grad``` iremos limpar a memória zerando todos os parâmetros que mantivemos em cache durante o processo de otimização.

----

Vejamos o exemplo de implementação para nosso bloco Linear, que é nossa matriz de pesos:

```py
class Linear(AiriLayer):

    def __init__(self, input_size=3072, hidden_size=10, reg=1e3, bias = True):
        self.config = None
        self.config_b = None
        self.bias = bias
        std = 1./ math.sqrt(input_size)
        self.w =  np.random.randn(input_size, hidden_size) * std
        self.b = np.zeros(hidden_size) if bias else None
        self.reg = reg
    
    def __call__(self, x):
        return self.forward(x)
    
    def forward(self, x, grad = True):
        if grad: 
            self.x = x
        return x@self.w + self.b
    
    def backward(self, dout):
        self.dW = self.x.T@dout
        self.dB = dout.sum(axis=0) if self.bias else None
        self.dW += self.reg * 2 * self.w 
        return dout@self.w.T 
        
    def update(self, lr=1e-3):
        self.w -= lr * self.dW
        self.b -= lr * self.dB
    
    def zero_grad(self):
        self.dW = None
        self.dB = None
        self.x = None
```

Veja que no construtor, inicializamos a matriz normalmente, como iremos treinar modelos maiores, utilizaremos a Kaiming Initialization. Perceba que simplesmente modularizamos o processo que construimos no Softmax Classifier, de maneira a deixar isto incrementável/escalável.

Para nossa layer Softmax:

```py
class Softmax(AiriLayer):

    def __init__(self, loss: str = "NLL"):
        self.loss_function = loss
        
    def __call__(self, x):
        return self.forward(x)

    def forward(self, scores, grad = True):
        self.batch_size = scores.shape[0]
        scores -= np.max(scores, axis=1, keepdims=True)
        scores_exp = np.exp(scores)
        softmax_matrix = scores_exp / np.sum(scores_exp, axis=1, keepdims=True) 
        if grad:
            self.softmax_matrix = softmax_matrix
        return softmax_matrix
    
    def NLLloss(self, y):
        loss = np.sum(-np.log(self.softmax_matrix[np.arange(self.batch_size), y]))
        loss /= self.batch_size
        return loss
    
    def backward(self, y):
        if self.loss_function == "NLL":
            self.softmax_matrix[np.arange(self.batch_size) ,y] -= 1
            self.softmax_matrix /= self.batch_size
            return self.softmax_matrix
        raise NotImplementedError("Unsupported Loss Function")
    
    def zero_grad(self):
        self.softmax_matrix = None
```

Novamente, não há nada de novo, o código aqui presente é copiado e colado do código que trabalhamos anteriormente.


----
O treinamento do Softmax Classifier agora é reduzido a:
```py
from layers import Linear, softmax

model = [
    Linear(input_size=3072, hidden_size=10, reg=1e3, bias=True),
    Softmax()
    ]

n_epoches = 500
x = train_images
y = train_targets

for i in range(n_epoches):
    
    for layer in model:
        x = layer.forward(x) # Não é necessario chamar o método forward. layer() é suficiente
    
    loss = model[-1].NLLloss(y)
    dout = model[-1].backward(y)
    
    for layer in reversed(model[:-1]):
        dout = layer.backward(dout)
        layer.update()
        layer.zero_grad()
```

Da mesma maneira, se quisermos agora treinar nossa Two Layer Neural Net, basta cria-la com blocos:

```py
model = [
    Linear(input_size=3072, hidden_size=100, reg=1e3, bias=True),
    Relu()
    Linear(input_size=100, hidden_size=10, reg=1e3, bias=True),
    Softmax()
]
```

E basta treinar da mesma maneira.


# Convoluções

TODO: Explain convs, I've done that too many times and Im not feeling in the mood to create visuals today.
